# Uso de Técnicas Multivariantes en Datos sobre Enfermedades Coronarias


#### La cardiopatÍa isquémica y otras enfermedades cardiovasculares son una de las principales causas de muerte a nivel global. Este trabajo parte de una base de datos real recogida en Framingham de más de 4000 registros y 16 variables. La variable respuesta es binaria y representa si el individuo del que se han obtenido los datos sufrirá cardiopatía isquémica en los 10 años siguientes. Se utilizan técnicas multivariantes no supervisadas (PCA y Clustering), supervisadas (árboles de decisión y SVM) y regresión logit para predecir la variable respuesta. Otro objetivo principal es comparar los modelos por sexo. Las conclusiones del estudio son la mejor predicción para las mujeres y unos resultados similares para todos los algoritmos realizados.

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sn
import scipy.stats as st
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import plotly

## PREPROCESAMIENTO

In [ ]:
# Se importan los datos y se le añade una columna con constante
# para la regresión que se utilizará más adelante.
df=pd.read_csv("framingham.csv")
df = df.dropna()
df.info()
col = list(df.columns)
col2= ['constante', col]
columns_names = df.columns.values

In [ ]:
# Se muestran los histogramas de las variables a estudiar.
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='royalblue')
        ax.set_title(feature+" Distribution", color='black')
    fig.tight_layout()
    plt.show()
draw_histograms(df, df.columns, 6, 3)

#### Se ven muchas variables dummies, entre ellas la variable objetivo.  Sobre esta variable se volverá tras el estudio descriptivo, puesto que tiene demasiados valores nulos respecto a los positivos.

In [ ]:
 # Como los datos tienen escalas muy diferentes, se escalan.
scaler = MinMaxScaler()
print(scaler.fit(df))
df_scaled = scaler.transform(df)
from statsmodels.tools import add_constant as add_constant
df_const = add_constant(df_scaled)
df_const = pd.DataFrame(df_const)
df_const.describe()

In [ ]:
# Se añaden los nombres de las columnas y se realiza la matriz de correlación.
fig, ax = plt.subplots(figsize=(15,12))
sn.heatmap(df_const.corr(), vmin=-1, vmax=1, annot=True, cmap="Blues", linewidths=.5, ax=ax)


#### Se va a realizar regresión logística como parte del procesamiento, puesto que dependiendo de la importancia de las variables estas serán candidatas a eliminarse del modelo.

In [ ]:
st.chisqprob = lambda chisq, df_const: st.chi2.sf(chisq, df)
x = df_const.iloc[:,0:15]
y = df.iloc[:,15]
x=x.to_numpy()
y = y.to_numpy()
model=sm.Logit(endog=y, exog=x)
result=model.fit()
model
result.summary()

In [ ]:
df_const = df_const.iloc[:,1:]
df_const.columns = col

In [ ]:
# La variables significactivas son:
df1=df_const[['age','male','cigsPerDay','totChol','sysBP','glucose','TenYearCHD']]
x=df1.iloc[:,:-1]
y=df1.iloc[:,-1]

#### La variable sexo es significativa y debido a las diferencias biologics que existen, a partir de ahora se separará la base de datos en hombres y mujeres para hacer los algoritmos por separado.


In [ ]:
df1 = df1.apply(pd.to_numeric)
H = df1[df1['male']>0]
H = H.drop(columns='male')
M = df1[df1['male']<1]
M = M.drop(columns='male')

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(8, 4), gridspec_kw=dict(width_ratios=[3, 3]))
sn.countplot(x='TenYearCHD',data=H, ax=axs[0])
sn.countplot(x='TenYearCHD',data=M,ax=axs[1])
f.tight_layout()

#### Se ha podido comprobar que existen demasiados resultados sin enfermedad coronaria para poder realizar un algoritmo de clasificación y que este no discrimine todo como 'NO'.
#### Se va a realizar un muestreo aleatorio de los registros sin enfermedad coronaria para crear una nueva base de datos con un número similar de registros negativos y positivos. Esto se realizará cuando se realicen los métodos supervisados.


## MÉTODOS NO SUPERVISADOS

### ANÁLISIS DE COMPONENTES PRINCIPALES

In [ ]:
from sklearn.decomposition import PCA

# Los dos df 'numer' se corresponden con el original pero el primero solo contiene las var significativas.
numer = df[['age','cigsPerDay','BPMeds','totChol','sysBP','diaBP','BMI','heartRate','glucose','TenYearCHD']]
# numer = df[['male','age','education','currentSmoker','cigsPerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','diaBP','BMI','heartRate','glucose','TenYearCHD']]

scaler = StandardScaler() # Se va a estandarizar las variables para darle la misma importancia en el modelo.
# df = numer.drop(['TenYearCHD'], axis=1) # Se elimina la variable objetivo, solo ejecutar una vez.
scaler.fit(numer) # Se calcula la media para poder hacer la transformacion
num_scal = scaler.transform(numer)
pca = PCA()
pca.fit(num_scal)
x_pca = pca.transform(num_scal)
expl = pca.explained_variance_ratio_
print(expl)

In [ ]:
# Tras observar la varianza explicada de los componentes, vemos la misma de los 5 primeros:
print('suma:',sum(expl[0:5]))
# Gráfico del codo
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.show()

In [ ]:
# Gráfico en dos dimensiones, tomando los dos primeros componentes principales
Xax=x_pca[:,0]
Yax=x_pca[:,1]
labels=numer['TenYearCHD'].values
cdict={0:'royalblue',1:'orange'}
labl={0:'Sin Riesgo',1:'Con Riesgo'}
marker={0:'o',1:'o'}
alpha={0:.3, 1:.3}
fig,ax=plt.subplots(figsize=(7,5))
fig.patch.set_facecolor('white')
for l in np.unique(labels):
    ix=np.where(labels==l)
    ax.scatter(Xax[ix],Yax[ix],c=cdict[l],label=labl[l],s=40,marker=marker[l],alpha=alpha[l])
 
plt.xlabel("Primer Componente Principal",fontsize=14)
plt.ylabel("Segundo Componente Principal",fontsize=14)
plt.show()

In [ ]:
# Gráfico de los 3 primeros Componentes principales
Zax=x_pca[:,2]
df['Zax'] = Zax
cdict={0:'royalblue',1:'orange'}
labl={0:'Sin Riesgo',1:'Con Riesgo'}
y = df.TenYearCHD

In [ ]:
# instalar matplotlib y ipympl
# %matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
# creating figure
fig = plt.figure()
ax = Axes3D(fig)
for l in np.unique(y):
 ix=np.where(y==l)
 ax.scatter(Xax[ix], 
            Yax[ix], 
            Zax[ix], 
            c=cdict[l], 
            s=12,
           label=labl[l])
  
ax.set_title("3D plot")
ax.set_xlabel('Primer Componente')
ax.set_ylabel('Segundo Componente')
ax.set_zlabel('Tercer Componente')
plt.show()

### CLUSTERING

#### K-MEANS

In [ ]:
from sklearn.cluster import KMeans
fig = plt.figure()
nc = range(1, 30) # El número de iteraciones que queremos hacer.
kmeans = [KMeans(n_clusters=i) for i in nc]
score = [kmeans[i].fit(numer).score(numer) for i in range(len(kmeans))]
score
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Suma de los errores cuadráticos')
plt.plot(nc,score)
# Gracias a este método, podemos ver que el número óptimo de conjuntos es 5. Se representará
# en 3 dimensiones utilizando PCA y se mostrarán número de conglomerados n=2 y n=5

In [ ]:
Xax=x_pca[:,0]
Yax=x_pca[:,1]
cdict={0:'royalblue',1:'orange'}
labl={0:'Sin Riesgo',1:'Con Riesgo'}
from matplotlib import pyplot
kmeans = KMeans(n_clusters = 2, init = 'k-means++', random_state = 42)
kmeans.fit(df_scaled)
y = kmeans.labels_

fig = plt.figure()
ax = Axes3D(fig)
for l in np.unique(y):
 ix=np.where(y==l)
 ax.scatter(Xax[ix], 
            Yax[ix], 
            Zax[ix], 
            c=cdict[l], 
            s=12,
           label=labl[l])
pyplot.show()

In [ ]:
cdict={0:'royalblue',1:'orange', 2:'lime', 3:'magenta'}
from matplotlib import pyplot
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
kmeans.fit(df_scaled)
y = kmeans.labels_

fig = plt.figure()
ax = Axes3D(fig)
for l in np.unique(y):
 ix=np.where(y==l)
 ax.scatter(Xax[ix], 
            Yax[ix], 
            Zax[ix], 
            c=cdict[l], 
            s=12)
pyplot.show()

#### AGLOMERATIVO

#### Este método escoge automáticamente el número de conglomerados óptimo.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering().fit(df_scaled)
y = clustering.labels_
cdict={0:'royalblue',1:'orange', 2:'lime', 3:'magenta'}

fig = plt.figure()
ax = Axes3D(fig)
for l in np.unique(y):
 ix=np.where(y==l)
 ax.scatter(Xax[ix], 
            Yax[ix], 
            Zax[ix], 
            c=cdict[l], 
            s=12)
pyplot.show()

## REGRESIÓN LOGÍSTICA

In [ ]:
# Se escoge el conjunto de entrenamiento del 70% del total de registros
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=7)

In [ ]:
def reglog (x_train, y_train, x_test, y_test):
    algoritmo = LogisticRegression()
    algoritmo.fit(x_train, y_train)
    y_pred = algoritmo.predict(x_test)
    #print('Summary del modelo')
    #print(algoritmo.summary())
    #Matriz de Confusión
    matriz = confusion_matrix(y_test, y_pred)
    print('Matriz de Confusión:')
    print(matriz)
    #Calculo la precisión del modelo
    precision = precision_score(y_test, y_pred)
    print('Precisión del modelo:')
    print(precision)
    #Calculo la exactitud del modelo
    exactitud = accuracy_score(y_test, y_pred)
    print('Exactitud del modelo:')
    print(exactitud)
    tn, fp, fn, tp = matriz.ravel()
    especificidad = tn / (tn+fp)
    print('Especificidad del modelo')
    print(especificidad)
    sensibilidad = recall_score(y_test, y_pred)
    print('Sensibilidad del modelo:')
    print(sensibilidad)
    #Calculo el Puntaje F1 del modelo
    puntajef1 = f1_score(y_test, y_pred)
    print('Puntaje F1 del modelo:')
    print(puntajef1)
    #Resumen
    print(classification_report(y_pred, y_test))
    #Calculo la curva ROC - AUC del modelo
    roc_auc = roc_auc_score(y_test, y_pred)
    print('Curva ROC - AUC del modelo:')
    print(roc_auc)
"""
    Gráfico de la curva ROC
    y_pred_proba = algoritmo.predict_proba(x_test)[::,1]
    from sklearn import metrics
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="auc="+str(auc))
    plt.legend(loc=4)
    plt.show()
"""
reglog(x_train, y_train, x_test, y_test)

#### Ya se ha visto que la única variable significativa que es dummy es el sexo. Se ha decidido realizar distintos modelos por sexo. Ahora se realizará por sexos y se equilibrrá el número de casos positivos y negativos en ambos sexos.

In [ ]:
# Se vrean conjuntos de entrenamiento y de test por sexo
Hx=H.iloc[:,:-1]
Hy=H.iloc[:,-1]
Mx=M.iloc[:,:-1]
My=M.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(Mx,My,test_size=.3,random_state=7)

In [ ]:
# Se equilibra el número de caoss en ambos sexos
shuffled_df_H = H.sample(frac=1,random_state=4)
CHD_df_H = shuffled_df_H.loc[shuffled_df_H['TenYearCHD'] == 1]
non_CHD_df_H = shuffled_df_H.loc[shuffled_df_H['TenYearCHD'] == 0].sample(n=350,random_state=42)
H_equi = pd.concat([CHD_df_H, non_CHD_df_H])
H_equi.TenYearCHD.value_counts()
sn.countplot(H_equi.TenYearCHD)
plt.box(False)
plt.xlabel('Enfermedad Coronaria No/Sí',fontsize=11)
plt.ylabel('Recuento',fontsize=11)
plt.title('Recuento de casos tras equilibrado para Hombres\n')
#plt.savefig('Balance Heart Disease.png')
plt.show()

H_equi_x=H_equi.iloc[:,:-1]
H_equi_y=H_equi.iloc[:,-1]

In [ ]:
shuffled_df_M = M.sample(frac=1,random_state=4)
CHD_df_M = shuffled_df_M.loc[shuffled_df_M['TenYearCHD'] == 1]
non_CHD_df_M = shuffled_df_M.loc[shuffled_df_M['TenYearCHD'] == 0].sample(n=320,random_state=42)
M_equi = pd.concat([CHD_df_M, non_CHD_df_M])
M_equi.TenYearCHD.value_counts()
sn.countplot(M_equi.TenYearCHD)
plt.box(False)
plt.xlabel('Enfermedad Coronaria No/Sí',fontsize=11)
plt.ylabel('Recuento',fontsize=11)
plt.title('Recuento de casos tras equilibrado para Mujeres\n')
#plt.savefig('Balance Heart Disease.png')
plt.show()

M_equi_x = M_equi.iloc[:,:-1]
M_equi_y = M_equi.iloc[:,-1]

In [ ]:
# Se crean los conjuntos de entrenamiento y test equilibrados por sexo.
H_equi_x_train, H_equi_x_test, H_equi_y_train, H_equi_y_test = train_test_split(H_equi_x,H_equi_y,test_size=.3,random_state=7)
M_equi_x_train, M_equi_x_test, M_equi_y_train, M_equi_y_test = train_test_split(M_equi_x,M_equi_y,test_size=.3,random_state=7)

#Tras haber definido la función, se ejecuta con 
reglog(H_equi_x_train, H_equi_y_train, H_equi_x_test, H_equi_y_test)
print("\n --------------------------------------------------------------------------------- \n")
reglog(M_equi_x_train, M_equi_y_train, M_equi_x_test, M_equi_y_test)

## ANOVA
#### Se realizan test ANOVA para diferentes varaibles entre sexos. También se realizan histogramas y boxplot para el estudio de las variables.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols


# Modelo OLS para distintas variables, en este caso 'totChol'
model = ols('totChol ~ C(male)', data=df1).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

sn.set(color_codes =True)
sn.distplot(H['sysBP'], rug=True)
sn.distplot(M['sysBP'], rug=True)
plt.show()

data = (H['sysBP'], M['sysBP'])
plt.boxplot(data)
plt.title("Boxplot Hombres y Mujeres")
plt.show()

## MÉTODOS SUPERVISADOS
#### La regresión logística realizada antes se podría considerar ya método supervisado, pero al utilizarla para la separación por sexos se ha decidido utilizarla por separado.

### ÁRBOLES DE DECISIÓN

#### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def decision_tree (x_train, y_train, x_test, y_test):
        #### RANDOM FOREST ####
    algoritmo = RandomForestClassifier()
    modelo = RandomForestClassifier(criterion= 'entropy',max_depth=2, random_state=0)
    
        #### CART ####
    #modelo = DecisionTreeClassifier(random_state=7)
    #modelo = DecisionTreeClassifier(min_samples_split=40, random_state=7)
    #modelo = DecisionTreeClassifier(criterion='gini', max_depth=7)
    #modelo = DecisionTreeClassifier(criterion='entropy', max_depth=7)
    #Mejora sensibilidad pero empeora especificidad
    
    modelo.fit(x_train, y_train)
    y_pred_dt = modelo.predict(x_test)
    y_pred = modelo.predict(x_test)
    dt_train_score = modelo.score(x_train, y_train)
    print('Clasificación en el cjto de entrenamiento %f' %dt_train_score)
    dt_accuracy = accuracy_score(y_pred_dt, y_test)
    print('Exactitud del modelo %f'%dt_accuracy)
    matriz = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6,5))
    sn.heatmap(pd.DataFrame(matriz), annot=True, cmap="Blues", ax=ax)
    plt.show()
    #Calculo la precisión del modelo
    precision = precision_score(y_test, y_pred)
    print('Precisión del modelo:')
    print(precision)
    #Calculo la exactitud del modelo
    exactitud = accuracy_score(y_test, y_pred)
    print('Exactitud del modelo:')
    print(exactitud)
    tn, fp, fn, tp = matriz.ravel()
    especificidad = tn / (tn+fp)
    print('Especificidad del modelo')
    print(especificidad)
    sensibilidad = recall_score(y_test, y_pred)
    print('Sensibilidad del modelo:')
    print(sensibilidad)
    #Calculo el Puntaje F1 del modelo
    puntajef1 = f1_score(y_test, y_pred)
    print('Puntaje F1 del modelo:')
    print(puntajef1)
    print(classification_report(y_pred, y_test))

decision_tree(M_equi_x_train, M_equi_y_train, M_equi_x_test, M_equi_y_test)

### SUPPORT VECTOR MACHINE
#### Para este algoritmo, podemos utilizar la función GirdSearch, que elije los parámetros òptimos del algoritmo para el conjunto de entrenamiento.

In [ ]:
from sklearn.svm import SVC, NuSVC
from sklearn.model_selection import GridSearchCV
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
svm_clf = GridSearchCV(SVC(kernel='linear', probability=True), param_grid, cv=10)

svm_clf.fit(H_equi_x_train, H_equi_y_train)
print("Para hombres, los mejores parámetros son:", svm_clf.best_params_ )
svm_clf.fit(M_equi_x_train, M_equi_y_train)
print("Para mujeres, los mejores parámetros son:", svm_clf.best_params_ )

In [ ]:
def SupportVM (x_train, y_train, x_test, y_test):
   # modelo=SVC(kernel='rbf')
    modelo=SVC(C=1, kernel='linear', gamma=0.001)
    modelo.fit(x_train, y_train)
    y_pred_dt = modelo.predict(x_test)
    dt_train_score = modelo.score(x_train, y_train)
    y_pred = modelo.predict(x_test)
    print('Clasificación en el cjto de entrenamiento %f' %dt_train_score)
    dt_accuracy = accuracy_score(y_pred_dt, y_test)
    matriz = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6,5))
    sn.heatmap(pd.DataFrame(matriz), annot=True, cmap="Blues", ax=ax)
    plt.show()
    print('Matriz de Confusión:')
    print(matriz)
    print('Exactitud del modelo %f'%dt_accuracy)
    #Calculo la precisión del modelo
    precision = precision_score(y_test, y_pred)
    print('Precisión del modelo:')
    print(precision)
    #Calculo la exactitud del modelo
    exactitud = accuracy_score(y_test, y_pred)
    print('Exactitud del modelo:')
    print(exactitud)
    tn, fp, fn, tp = matriz.ravel()
    especificidad = tn / (tn+fp)
    print('Especificidad del modelo')
    print(especificidad)
    sensibilidad = recall_score(y_test, y_pred)
    print('Sensibilidad del modelo:')
    print(sensibilidad)
    #Calculo el Puntaje F1 del modelo
    puntajef1 = f1_score(y_test, y_pred)
    print('Puntaje F1 del modelo:')
    print(puntajef1)
    print(classification_report(y_pred, y_test))
    #Calculo la curva ROC - AUC del modelo
    roc_auc = roc_auc_score(y_test, y_pred)
    print('Curva ROC - AUC del modelo:')
    print(roc_auc)
    
SupportVM(M_equi_x_train, M_equi_y_train, M_equi_x_test, M_equi_y_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing 

from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches

from tabulate import tabulate

def split_dataset(X, y, test_size = 0.25, random_state = 1):
    X_train, X_test, y_train, y_test=train_test_split(X, y, 
                                                      test_size = test_size, 
                                                      random_state = random_state,                                                   shuffle=True)
    return X_train, X_test, y_train, y_test

def draw_matrix_confusion(yTest, yPred, subplot, name, measures):
    LABELS = ['No Fraude', 'Fraude'] 
    conf_matrix = metrics.confusion_matrix(yTest, yPred) 
    sns.set(font_scale=2)
    ax = plt.subplot(3, 2, subplot)

    sns.heatmap(conf_matrix, xticklabels = LABELS,  
                yticklabels = LABELS, annot = True, fmt ="d"); 
    plt.title(name, fontsize=20, fontweight='bold') 
    plt.ylabel('Clase verdadera', fontsize=20, fontweight='bold') 
    plt.xlabel('Clase predicha\nROC-AUC: {}'.format(measures[-1]), fontsize=20, fontweight='bold') 
    return conf_matrix

# Define las medidas que se van a probar en cada algoritmo
def calculate_measures(y_test, yPred):
    acc = metrics.accuracy_score(y_test, yPred)
    prec = metrics.precision_score(y_test, yPred, average='micro')
    rec = metrics.recall_score(y_test, yPred, average='micro')
    f1 = metrics.f1_score(y_test, yPred, average='weighted', labels=np.unique(yPred))
    roc_auc = metrics.roc_auc_score(y_test, yPred)
    MCC = metrics.matthews_corrcoef(y_test, yPred)
    matriz = confusion_matrix(y_test, yPred)
    tn, fp, fn, tp = matriz.ravel()
    espec = tn / (tn+fp)
    return acc, prec, rec, f1, roc_auc, MCC, espec

# Define los algoritmos de clasificación
algs = [['Regresión Logística', LogisticRegression()],
    ['Árbol de Decisión (Criterion Gini)', DecisionTreeClassifier(criterion='gini')],
    ['Árbol de Decisión (Criterion Entropy)', DecisionTreeClassifier(criterion='entropy')],
    ['Random Forest', RandomForestClassifier()],
    ['Máquinas de Vectores de Soporte - Lineal', SVC(kernel='linear')],
    ['Máquinas de Vectores de Soporte - RBF', SVC(kernel='rbf')]
]


def make_machine_learning(X, y, classifier, auto_enc = False):
    
    X_train, X_test, y_train, y_test = split_dataset(X, y, random_state=1, test_size = 0.25)

    classifier.fit(X_train, y_train)
    yPred = classifier.predict(X_test)

    measures = calculate_measures(y_test, yPred)

    return X_train, X_test, y_train, y_test, measures, yPred

def train(X, y):
    fig = plt.figure(figsize =(26, 20))
    fig.suptitle('Matrices de confusion', fontsize=30, fontweight='bold')
    performance_measures = []   
    i = 1
    for name, classifier in algs:
        tup = []
        print('Entrenando modelo con la tecnica ' + name)
        X_train, X_test, y_train, y_test, measures, yPred = make_machine_learning(X, y, classifier)
        tup.append(name)
        for mea in measures:
            tup.append(mea)
        performance_measures.append(tup)
        print('Dibujando matrices de confusion para ' + name)
        confusion_matrix = draw_matrix_confusion(y_test, yPred, i, name, measures)
        i += 1
    print('\n')
    fig.tight_layout(pad=3.0)
    plt.show()
    print('')
    print(tabulate(performance_measures, headers=['Método', 'Exactitud', 'Precisión', 'Sensibilidad', 'Puntaje F1', 'Curva ROC AUC', 'MCC', 'Especificidad']))

train(M_equi_x, M_equi_y)

In [ ]:
train(H_equi_x, H_equi_y)